# 문제 1. 
- 다음은 Customer Segmentation 데이터 세트이다. 주어진 훈련 데이터 세트를 활용하여 고객이 속한 세그먼트(Segmentation)를 예측하고 해당 예측 결과를 csv 파일로 제출하시오.
- 결과 제출 양식 : 제출한 예측값의 macro_f1 결과를 통해 영역별 배점에 따라 최종 점수가 반영될 예정

In [1]:
import pandas as pd 
pd.DataFrame({
    "ID"  : [1, 2, 3],
    "y_pred" : ["A", "B", "C"]
})

,ID,y_pred
0,1,A
1,2,B
2,3,C


## 데이터 불러오기

In [2]:
import pandas as pd 
X_test = pd.read_csv("data/4회/404_x_test.csv")
X_train = pd.read_csv("data/4회/404_x_train.csv")
y_train = pd.read_csv("data/4회/404_y_train.csv")

In [3]:
import pandas as pd 
pd.DataFrame({
    "변수" : list(X_train.columns) + ["Segmentation"], 
    "설명" : [
        "고객 ID 번호", 
        "성별", 
        "결혼 여부", 
        "나이", 
        "대학 졸업 여부", 
        "직업",
        "근무 연수", 
        "지출 수준", 
        "가족 수(본인 포함)",   
        "고객 세그먼트(A, B, C, D 중 하나)"
    ]
})

,변수,설명
0,ID,고객 ID 번호
1,Gender,성별
2,Ever_Married,결혼 여부
3,Age,나이
4,Graduated,대학 졸업 여부
5,Profession,직업
6,Work_Experience,근무 연수
7,Spending_Score,지출 수준
8,Family_Size,가족 수(본인 포함)
9,Segmentation,"고객 세그먼트(A, B, C, D 중 하나)"


## 데이터 탐색

In [4]:
X_train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0
1,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0
2,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0
3,461319,Male,Yes,56,No,Artist,0.0,Average,2.0
4,460156,Male,No,32,Yes,Healthcare,1.0,Low,3.0


In [5]:
y_train.head()

,ID,Segmentation
0,462809,D
1,466315,B
2,461735,B
3,461319,C
4,460156,C


In [6]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6718 entries, 0 to 6717
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               6718 non-null   int64  
 1   Gender           6718 non-null   object 
 2   Ever_Married     6718 non-null   object 
 3   Age              6718 non-null   int64  
 4   Graduated        6718 non-null   object 
 5   Profession       6718 non-null   object 
 6   Work_Experience  6718 non-null   float64
 7   Spending_Score   6718 non-null   object 
 8   Family_Size      6718 non-null   float64
dtypes: float64(2), int64(2), object(5)
memory usage: 472.5+ KB


In [7]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6718 entries, 0 to 6717
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            6718 non-null   int64 
 1   Segmentation  6718 non-null   object
dtypes: int64(1), object(1)
memory usage: 105.1+ KB


In [8]:
len(X_train['ID'].unique())

6718

In [9]:
len(y_train['ID'].unique())

6718

In [10]:
X_train.isnull().sum()

ID                 0
Gender             0
Ever_Married       0
Age                0
Graduated          0
Profession         0
Work_Experience    0
Spending_Score     0
Family_Size        0
dtype: int64

In [11]:
y_train.isnull().sum()

ID              0
Segmentation    0
dtype: int64

In [12]:
X_train.describe()

,ID,Age,Work_Experience,Family_Size
count,6718.000000,6718.000000,6718.000000,6718.000000
mean,463516.571152,43.524263,2.629652,2.841768
std,2566.017254,16.513115,3.404639,1.523319
min,458982.000000,18.000000,0.000000,1.000000
25%,461347.250000,31.000000,0.000000,2.000000
50%,463566.000000,41.000000,1.000000,2.000000
75%,465739.750000,53.000000,4.000000,4.000000
max,467974.000000,89.000000,14.000000,9.000000


In [13]:
y_train['Segmentation'].value_counts()

D    1772
C    1735
A    1628
B    1583
Name: Segmentation, dtype: int64

## 데이터셋 분리

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5374, 9), (1344, 9), (5374, 2), (1344, 2))

## 컬럼 분리

In [15]:
X_test_id = X_test.pop('ID')
X_train.pop('ID')
y_test.pop('ID')
y_train.pop('ID')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5374, 8), (1344, 8), (5374, 1), (1344, 1))

## 데이터 타입별로 분리

In [16]:
object_df = X_train.select_dtypes(include=object)
object_df.head()

,Gender,Ever_Married,Graduated,Profession,Spending_Score
2481,Male,Yes,Yes,Homemaker,Average
1253,Male,Yes,No,Entertainment,Low
5918,Female,No,No,Healthcare,Low
5744,Female,Yes,No,Doctor,High
6151,Male,Yes,Yes,Entertainment,Low


In [17]:
import numpy as np

number_df = X_train.select_dtypes(include=np.number)
number_df.head()

,Age,Work_Experience,Family_Size
2481,35,9.0,4.0
1253,50,0.0,2.0
5918,21,1.0,7.0
5744,51,1.0,1.0
6151,43,0.0,2.0


In [18]:
for column in object_df.columns:
    print(object_df[column].value_counts())
    print()

Male      2970
Female    2404
Name: Gender, dtype: int64

Yes    3170
No     2204
Name: Ever_Married, dtype: int64

Yes    3424
No     1950
Name: Graduated, dtype: int64

Artist           1764
Healthcare        882
Entertainment     648
Doctor            483
Engineer          465
Executive         403
Lawyer            400
Marketing         190
Homemaker         139
Name: Profession, dtype: int64

Low        3239
Average    1323
High        812
Name: Spending_Score, dtype: int64



## 모델생성

In [36]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, LabelEncoder, OneHotEncoder, OrdinalEncoder
from lightgbm import LGBMClassifier

# 컬럼변환기
transformer = ColumnTransformer([
    ('minmax_scaler', MinMaxScaler(), number_df.columns),
    ('ohc_encoder', OneHotEncoder(), ['Gender', 'Ever_Married', 'Graduated', 'Profession']),
    ('ordinal_encoder', OrdinalEncoder(categories=[['Low', 'Average', 'High']]), ['Spending_Score'])
], remainder='passthrough')

# 파이프라인
pipeline = Pipeline([
    ('preprocessor', transformer),
    ('clf', LGBMClassifier(random_state=42, max_depth=5, learning_rate=0.02))
])

# 모델학습
pipeline.fit(X_train, y_train)

D:\Anaconda\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('minmax_scaler',
                                                  MinMaxScaler(),
                                                  Index(['Age', 'Work_Experience', 'Family_Size'], dtype='object')),
                                                 ('ohc_encoder',
                                                  OneHotEncoder(),
                                                  ['Gender', 'Ever_Married',
                                                   'Graduated', 'Profession']),
                                                 ('ordinal_encoder',
                                                  OrdinalEncoder(categories=[['Low',
                                                                              'Average',
                                                                              'High']]),
                                                  ['Spending_Score'])])),
                ('clf',
                 LGBMClassifier(learning_rate=0.02, max_depth=5,
                                random_state=42))])

## 모델평가

In [37]:
y_pred = pipeline.predict(X_test)
y_pred

array(['A', 'A', 'C', ..., 'C', 'A', 'A'], dtype=object)

In [38]:
from sklearn.metrics import f1_score

def get_scores(model, X_train, X_test, y_train, y_test):
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    train_score = f1_score(y_train, y_train_pred, average='macro')
    test_score = f1_score(y_test, y_test_pred, average='macro')
    return f'train : {round(train_score, 3)}, valid : {round(test_score, 4)}'

get_scores(pipeline, X_train, X_test, y_train, y_test)

'train : 0.557, valid : 0.5042'

## 결과제출

In [39]:
result = pd.DataFrame({'ID':X_test_id, 'y_pred':y_pred})
result

,ID,y_pred
5774,464716,A
4221,467301,A
4697,466937,C
1616,462323,C
1886,464706,D
...,...,...
4333,466808,A
422,464026,D
3355,466696,C
4814,463304,A


In [40]:
result.to_csv('3.csv', index=False)